In [1]:
#To print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [3]:
tokyo = pd.read_excel('Fellow Project - Tokyo Data.xlsx')

In [4]:
tokyo.head()

,Competition_Name,Competitor,Event_Gender,Event_ID,Event_Name,Sport_Name,NOC,NOC_Name,Person_ID,Result,Result_Type,Sport_ID,Team_ID,Team_Mbrs,Team_Member_Select,Person_Age_Days,Final Rank
0,Olympic Games,A. Murray/Salisbury,Men,100010265,Doubles,Tennis,GBR,Great Britain,-1,NaN,NaN,105,200140186,"Murray, Andy; Salisbury, Joe",No,NaN,5.0
1,Olympic Games,A'Ja Wilson,Women,200010983,Team,Basketball,USA,United States,1192461,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",Yes,NaN,1.0
2,Olympic Games,Aaliyah Edwards,Women,200010983,Team,Basketball,CAN,Canada,1828248,NaN,NaN,107,100000355,"Achonwa, Natalie; Alexander, Kayla; Amihere, L...",Yes,NaN,9.0
3,Olympic Games,Aaron Brown,Men,100009572,200m,Athletics,CAN,Canada,798318,20.2,Time,106,400000309,NaN,No,10661.0,6.0
4,Olympic Games,Aaron Brown,Men,100009572,200m,Athletics,CAN,Canada,798318,20.2,Time,106,400000309,NaN,Yes,10661.0,6.0


In [5]:
tokyo.shape

(38643, 17)

In [6]:
#Will drop duplicates
df = tokyo.drop_duplicates()

In [7]:
df.shape

(38643, 17)

In [8]:
#Checking null values to assess data quality
df.isna().sum()

Competition_Name          0
Competitor                0
Event_Gender              0
Event_ID                  0
Event_Name                0
Sport_Name                0
NOC                       0
NOC_Name                  0
Person_ID                 0
Result                13628
Result_Type           11071
Sport_ID                  0
Team_ID                   0
Team_Mbrs             27769
Team_Member_Select        0
Person_Age_Days       10879
Final Rank             1166
dtype: int64

### 1. Deleting Team_Member_Select column

##### The Team_Member_Select column is adding duplicate records in the dataset. 
##### Here, Aaron Brown's entries for the 200m event are duplicated, differing only in the Team_Member_Select value (No vs. Yes). This redundancy can lead to inaccurate analysis so we will remove this column.

In [9]:
df.iloc[3:5,]

,Competition_Name,Competitor,Event_Gender,Event_ID,Event_Name,Sport_Name,NOC,NOC_Name,Person_ID,Result,Result_Type,Sport_ID,Team_ID,Team_Mbrs,Team_Member_Select,Person_Age_Days,Final Rank
3,Olympic Games,Aaron Brown,Men,100009572,200m,Athletics,CAN,Canada,798318,20.2,Time,106,400000309,NaN,No,10661.0,6.0
4,Olympic Games,Aaron Brown,Men,100009572,200m,Athletics,CAN,Canada,798318,20.2,Time,106,400000309,NaN,Yes,10661.0,6.0


In [10]:
df.drop(columns='Team_Member_Select', inplace=True)

In [11]:
df.shape

(38643, 16)

In [12]:
df = df.drop_duplicates()

In [13]:
df.shape

(24759, 16)

In [23]:
#Dataset has shrunk considerably

## 2. Standardizing Person_Age_Days across common Person_IDs

##### The dataset shows inconsistency in the Person_Age_Days column for Person_ID 1184868. The age values vary or are missing across different records for the same person, as illustrated below 

In [14]:
df[df['Person_ID']==1184868][['Person_ID', 'Person_Age_Days']]

,Person_ID,Person_Age_Days
3664,1184868,9115.0
3666,1184868,9113.0
3668,1184868,NaN
3669,1184868,NaN
3670,1184868,9116.0
3672,1184868,NaN


##### To standardize this dataset, for each Person_ID, we will fill Person_Age_Days with first non-null value 

In [15]:
def first_non_null(series):
    return series.dropna().iloc[0] if not series.dropna().empty else np.nan

In [16]:
df['Person_Age_Days'] = df.groupby('Person_ID')['Person_Age_Days'].transform(first_non_null)

In [17]:
#to check functionality
df[df['Person_ID']==1184868][['Person_ID', 'Person_Age_Days']]

,Person_ID,Person_Age_Days
3664,1184868,9115.0
3666,1184868,9115.0
3668,1184868,9115.0
3669,1184868,9115.0
3670,1184868,9115.0
3672,1184868,9115.0


In [18]:
# For ease of calculation, we will convert person_age_days to person_age_years and round it to 0 decimal
df['Person_Age_Years'] = np.floor(df['Person_Age_Days'] / 365)

### 3. For team events, the final rank will be 1 for all team members but the medal given will be just 1. 

In [19]:
df[(df['Event_Name']=='Team') & (df['Final Rank']==1) & (df['Event_ID']==200010983)]

,Competition_Name,Competitor,Event_Gender,Event_ID,Event_Name,Sport_Name,NOC,NOC_Name,Person_ID,Result,Result_Type,Sport_ID,Team_ID,Team_Mbrs,Person_Age_Days,Final Rank,Person_Age_Years
1,Olympic Games,A'Ja Wilson,Women,200010983,Team,Basketball,USA,United States,1192461,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
2411,Olympic Games,Ariel Atkins,Women,200010983,Team,Basketball,USA,United States,1865314,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
3458,Olympic Games,Breanna Stewart,Women,200010983,Team,Basketball,USA,United States,958020,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
3546,Olympic Games,Brittney Griner,Women,200010983,Team,Basketball,USA,United States,1025574,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
4210,Olympic Games,Chelsea Gray,Women,200010983,Team,Basketball,USA,United States,1865479,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
5590,Olympic Games,Diana Taurasi,Women,200010983,Team,Basketball,USA,United States,558154,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
10332,Olympic Games,Jewell Loyd,Women,200010983,Team,Basketball,USA,United States,1500997,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
16572,Olympic Games,Napheesa Collier,Women,200010983,Team,Basketball,USA,United States,1303905,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
20687,Olympic Games,Skylar Diggins,Women,200010983,Team,Basketball,USA,United States,814217,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN
21110,Olympic Games,Sue Bird,Women,200010983,Team,Basketball,USA,United States,447561,NaN,NaN,107,100000499,"Atkins, Ariel; Bird, Sue; Charles, Tina; Diggi...",NaN,1.0,NaN


##### To ensure we get a correct count of medals won, we will create a new column

##### The assign_medal_count function sorts a group by 'Final Rank' and assigns a unique medal count to each unique rank, and sets duplicate ranks to 0. This ensures each unique rank is counted once in the 'Medal_count' column.

In [20]:
def assign_medal_count(group):
    group = group.sort_values(by='Final Rank')
    medal_count = []
    seen_ranks = set()
    for rank in group['Final Rank']:
        if rank not in seen_ranks:
            medal_count.append(rank)
            seen_ranks.add(rank)
        else:
            medal_count.append(0)
    group['Medal_count'] = medal_count
    return group

df_new = df.groupby(['Event_ID', 'Team_ID']).apply(assign_medal_count).reset_index(drop=True)

In [21]:
#checking the functionality
df_new[(df_new['Event_Name']=='Team') & (df_new['Final Rank']==1) & (df_new['Team_ID']==100021052)]

,Competition_Name,Competitor,Event_Gender,Event_ID,Event_Name,Sport_Name,NOC,NOC_Name,Person_ID,Result,Result_Type,Sport_ID,Team_ID,Team_Mbrs,Person_Age_Days,Final Rank,Person_Age_Years,Medal_count
18246,Paralympic Games,Alexis Shifflett,Women,200011562,Team,Sitting Volleyball,USA,United States,1274283,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,1.0
18247,Paralympic Games,Bethany Zummo,Women,200011562,Team,Sitting Volleyball,USA,United States,1065736,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18248,Paralympic Games,Emma Schieck,Women,200011562,Team,Sitting Volleyball,USA,United States,2206927,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18249,Paralympic Games,Heather Erickson,Women,200011562,Team,Sitting Volleyball,USA,United States,1035874,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18250,Paralympic Games,Jillian Kathleen Williams,Women,200011562,Team,Sitting Volleyball,USA,United States,1825264,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18251,Paralympic Games,Kaleo Maclay,Women,200011562,Team,Sitting Volleyball,USA,United States,1065533,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18252,Paralympic Games,Kathryn Holloway,Women,200011562,Team,Sitting Volleyball,USA,United States,1274377,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18253,Paralympic Games,Lora Webster,Women,200011562,Team,Sitting Volleyball,USA,United States,681496,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18254,Paralympic Games,Monique Burkland,Women,200011562,Team,Sitting Volleyball,USA,United States,1066175,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0
18255,Paralympic Games,Nicole Nieves,Women,200011562,Team,Sitting Volleyball,USA,United States,1274387,NaN,NaN,520,100021052,"Webster, Lora; Erickson, Heather; Maclay, Kale...",NaN,1.0,NaN,0.0


In [22]:
df_new.to_csv('clean_data.csv', index=False)